# UDC - Hackathon Urban Data Challenge 
## Emotions


### Ressources

In [35]:
import math
import numpy

x_pi = 3.14159265358979324*3000/180
pi = 3.1415926535897932384626
a = 6378245.0
ee = 0.00669342162296594323

def GCJ02toWGS84LNG(lng, lat):
   # if out_of_china(lng, lat):
    #    return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = numpy.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = numpy.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * numpy.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return lng * 2 - mglng

def GCJ02toWGS84LAT(lng, lat):
   # if out_of_china(lng, lat):
    #    return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = numpy.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = numpy.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * numpy.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return lat * 2 - mglat

def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
        0.1 * lng * lat + 0.2 * numpy.sqrt(numpy.fabs(lng))
    ret += (20.0 * numpy.sin(6.0 * lng * pi) + 20.0 *
            numpy.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * numpy.sin(lat * pi) + 40.0 *
            numpy.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * numpy.sin(lat / 12.0 * pi) + 320 *
            numpy.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret
 
def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
        0.1 * lng * lat + 0.1 * numpy.sqrt(numpy.fabs(lng))
    ret += (20.0 * numpy.sin(6.0 * lng * pi) + 20.0 *
            numpy.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * numpy.sin(lng * pi) + 40.0 *
            numpy.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * numpy.sin(lng / 12.0 * pi) + 300.0 *
            numpy.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret
 
def out_of_china(lng, lat):
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

In [36]:
from functools import reduce
def first_score(score):
    score = score.values[0].split(',')[0]
    #print (reduce(lambda x, y: x + y, score) / len(score))
    print(score)

### Import database

In [37]:
import sqlite3

#connection
#path = 'C:\Users\lolal\Documents\Shanghai\Hackathon\UTSEUS-shanghai-dianping.db'
conn = sqlite3.connect('UTSEUS-shanghai-dianping.db')
c = conn.cursor()

#request 
c.execute('SELECT name, latitude, longitude, avg_rating, category, avg_price, address FROM venues, venue_categories WHERE venues.business_id = venue_categories.business_id')
venues = c.fetchall()


In [38]:
conn = sqlite3.connect('UTSEUS-shanghai-weibo-2012.db')
c = conn.cursor()

c.execute('SELECT name, latitude, longitude, scores, category, price FROM dianping_shop')
shops = c.fetchall()

### Make Dataframe

In [39]:
import pandas as pd 
labelsD= ['name','latitude', 'longitude', 'scores','category', 'price', 'address']
labelsW = ['name','latitude', 'longitude', 'scores','category', 'price']

df_Dianping = pd.DataFrame.from_records(venues, columns=labelsD)
df_Weibo = pd.DataFrame.from_records(shops, columns=labelsW)


### Clean data
#### Dianping

In [40]:
# remove duplicate
df_Dianping = df_Dianping.drop_duplicates(subset=['name'], keep='first')

#concentrate on XuHui list
df_Dianping = df_Dianping[['徐汇区' in cat for cat in df_Dianping['address']]]



In [41]:
#change coordinates format GCJ02 to WGS84
df_Dianping['longitude'] = GCJ02toWGS84LNG(df_Dianping['longitude'], df_Dianping['latitude'])
df_Dianping['latitude'] = GCJ02toWGS84LAT(df_Dianping['longitude'], df_Dianping['latitude'])

df_Dianping= df_Dianping[(df_Dianping['latitude']>31) & (df_Dianping['longitude']>121)]

min_lng = df_Dianping['longitude'].min()
max_lng = df_Dianping['longitude'].max()
min_lat = df_Dianping['latitude'].min()
max_lat = df_Dianping['latitude'].max()

print(min_lat, min_lng, max_lat, max_lng )

31.00403221349404 121.16389285310815 31.655458578502852 121.79478477798658


In [42]:
#select the data
df_Dianping_final = df_Dianping.drop(['address'], axis=1)

In [43]:
df_Dianping_temp = df_Dianping_final[['熟食' in cat for cat in df_Dianping['category']]]
df_Dianping_temp = df_Dianping_temp.append(df_Dianping_final[['小吃' in cat for cat in df_Dianping['category']]])
df_Dianping_temp = df_Dianping_temp.append(df_Dianping_final[['寿司/简餐' in cat for cat in df_Dianping['category']]])
df_Dianping_temp = df_Dianping_temp.append(df_Dianping_final[['小吃快餐' in cat for cat in df_Dianping['category']]])
df_Dianping_temp = df_Dianping_temp.append(df_Dianping_final[['星巴克' in cat for cat in df_Dianping['category']]])
df_Dianping_temp = df_Dianping_temp.append(df_Dianping_final[['酒吧' in cat for cat in df_Dianping['category']]])

df_Dianping_final = df_Dianping_temp

In [44]:
min_lng = df_Dianping_final['longitude'].min()
max_lng = df_Dianping_final['longitude'].max()
min_lat = df_Dianping_final['latitude'].min()
max_lat = df_Dianping_final['latitude'].max()

print(min_lat, min_lng, max_lat, max_lng )

31.02523926183565 121.3901985611001 31.310136303299014 121.50520099657118


In [45]:
#get the categories
c_Dianping = df_Dianping.groupby('category').count()
c_Dianping = c_Dianping[['name']]
c_Dianping.columns = ['number']

#### Weibo

In [46]:
# remove duplicate
df_Weibo = df_Weibo.drop_duplicates(subset=['name'], keep='first')

#concentrate on XuHui list
#no district in the address -> with dianping coo
#df_Weibo = df_Weibo[['徐汇区' in cat for cat in df_Weibo['address']]]


In [47]:
#change coordinates format GCJ02 to WGS84
df_Weibo['longitude'] = GCJ02toWGS84LNG(df_Weibo['longitude'], df_Weibo['latitude'])
df_Weibo['latitude'] = GCJ02toWGS84LAT(df_Weibo['longitude'], df_Weibo['latitude'])


In [48]:
#select the data
df_Weibo_final = df_Weibo[['shop' in cat for cat in df_Weibo['category']]]
df_Weibo_final = df_Weibo_final[(df_Weibo_final['longitude'] < max_lng) & (df_Weibo_final['longitude'] > min_lng) & (df_Weibo_final['latitude'] < max_lat) & (df_Weibo_final['latitude'] > min_lat)]


In [49]:
for i in range(19367):
    df_Weibo_final['scores'].values[i] = df_Weibo_final['scores'].values[i].split(',')[0]
    

In [50]:
#get the categories
c_Weibo = df_Weibo.groupby('category').count()
c_Weibo = c_Weibo[['name']]
c_Weibo.columns = ['number']

### Make csv

In [57]:
c_Dianping.to_csv('UDC18\c_Dianping.csv', sep='\t', encoding='utf-8')

In [58]:
c_Weibo.to_csv('UDC18\c_Weibo.csv', sep='\t', encoding='utf-8')

In [59]:
df_Weibo_final.to_csv('UDC18\df_Weibo.csv', sep='\t', encoding='utf-8')

In [60]:
df_Dianping_final .to_csv('UDC18\df_Dianping.csv', sep='\t', encoding='utf-8')

In [61]:
df_final = df_Weibo_final 
df_final = df_final.append(df_Dianping_final, sort=True)


In [62]:
df_final.to_csv('UDC18\df_final.csv', sep='\t', encoding='utf-8')